# DESAFIO 2 PNL
# Procesamiento de lenguaje natural
## Custom embedddings con Gensim

Alumna: Sofia Speri

Objetivo:
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.



### Librerías


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

import os
import platform

### Dataset
Voy a utilizar dataset sobre FAQ's medical question-answer pairs created from 12 NIH websites (e.g. cancer.gov, niddk.nih.gov, GARD, MedlinePlus Health Topics). Fuente: https://www.kaggle.com/datasets/jpmiller/layoutlm

Del dataset tomé un grupo de preguntas y respuestas y las junté en un sólo archivo con formato txt para este trabajo.

In [2]:
df = pd.read_csv('/content/cancer_faqs.txt', sep='/n', header=None)
df.head()

<ipython-input-2-091af6ed99d1>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/cancer_faqs.txt', sep='/n', header=None)


,0
0,What is are Adult Acute Myeloid Leukemia ?
1,- Adult acute myeloid leukemia (AML) is a type...
2,Who is at risk for Adult Acute Myeloid Leukemi...
3,"Smoking, previous chemotherapy treatment, and ..."
4,What are the symptoms of Adult Acute Myeloid L...


In [3]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 484


### 1 - Preprocesamiento

#### Eliminación de "Stop Words"
Uso la función "remove_stopwords" de gensim para eliminar las palabras que no son significativas en los documentos como conectores, advervios, etc

In [4]:
from gensim.parsing.preprocessing import remove_stopwords

df[0] = df[0].apply(lambda x: remove_stopwords(x))

In [5]:
#Verificación
df.head()

,0
0,What Adult Acute Myeloid Leukemia ?
1,- Adult acute myeloid leukemia (AML) type canc...
2,Who risk Adult Acute Myeloid Leukemia? ?
3,"Smoking, previous chemotherapy treatment, expo..."
4,What symptoms Adult Acute Myeloid Leukemia ?


#### Oraciones en secuencias de palabras

In [6]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []

for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

### 2 -Vectores Word2vec

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

#### Modelo generador de vectores.

Elegí los siguientes hiperparámetros que dieron mejores resultados:

Hiperparámetros elegidos:
- Frecuencia de repetición mayor a 5 : dado que el texto es bastante específico y ciertos conceptos pueden estar en algunas preguntas/respuestas; veo que la frecuencia de palabras puede no ser tan alta

- Ventana de 10 palabras antes y después: para que capture mejor ciertas relaciones y conceptos

- Vector de embedding de largo 200: Para este caso con los hiperparámetros elegidos no hubo tanta diferencia en cuanto a valor de loss y resultados de ensayos y analogías; para embeddings de 200/300 y 400 valores, por lo que elijo el embedding más chico para optimización.



In [8]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,
                     window=10,
                     vector_size=200,
                     negative=20,
                     workers=1,
                     sg=1)

In [9]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [10]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 484


In [11]:
# Cantidad de words encontradas en el corpus
print("Cantidad de palabras distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de palabras distintas en el corpus: 1532


### 3 - Entrenar embeddings

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=30,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 2011376.125
Loss after epoch 1: 1544263.625
Loss after epoch 2: 1436886.25
Loss after epoch 3: 1377684.0
Loss after epoch 4: 1367587.5
Loss after epoch 5: 1308703.5
Loss after epoch 6: 1260645.0
Loss after epoch 7: 1252999.0
Loss after epoch 8: 1262982.0
Loss after epoch 9: 1256158.0
Loss after epoch 10: 1256721.0
Loss after epoch 11: 1261337.0
Loss after epoch 12: 1128223.0
Loss after epoch 13: 1112678.0
Loss after epoch 14: 1111588.0
Loss after epoch 15: 1113170.0
Loss after epoch 16: 1113124.0
Loss after epoch 17: 1109892.0
Loss after epoch 18: 1117548.0
Loss after epoch 19: 1118792.0
Loss after epoch 20: 1119366.0
Loss after epoch 21: 1119308.0
Loss after epoch 22: 1123414.0
Loss after epoch 23: 1119558.0
Loss after epoch 24: 1129098.0
Loss after epoch 25: 1127898.0
Loss after epoch 26: 1145850.0
Loss after epoch 27: 970278.0
Loss after epoch 28: 964880.0
Loss after epoch 29: 985080.0


(1886168, 2505630)

### 4 - Ensayos

Pruebo palabras más similares para 'Sangre', 'Mujer' y 'Gen'

In [13]:
w2v_model.wv.most_similar(positive=["blood"], topn=10)

[('vessels', 0.5052155256271362),
 ('serum', 0.4661736488342285),
 ('break', 0.4638121724128723),
 ('chemistry', 0.4534189999103546),
 ('hemoglobin', 0.44031837582588196),
 ('portion', 0.4377850294113159),
 ('platelets', 0.4313320219516754),
 ('contrast', 0.42520567774772644),
 ('cortisol', 0.4242464005947113),
 ('oxygen', 0.4199490547180176)]

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["women"], topn=10)

[('premenopausal', 0.5405682921409607),
 ('postmenopausal', 0.5168856382369995),
 ('breast', 0.5036075115203857),
 ('pregnant', 0.4978010356426239),
 ('nursing', 0.47688189148902893),
 ('woman', 0.4687138795852661),
 ('dense', 0.4482439160346985),
 ('men', 0.4474965035915375),
 ('feeding', 0.4455025792121887),
 ('reducing', 0.44507986307144165)]

In [15]:
vector_gene = w2v_model.wv.get_vector('gene')
w2v_model.wv.most_similar(positive=[vector_gene], topn=10)

[('gene', 1.0),
 ('mutations', 0.766743540763855),
 ('mutation', 0.7245761156082153),
 ('smarcb1', 0.5971249938011169),
 ('brca1', 0.5829885005950928),
 ('parent', 0.5802830457687378),
 ('smarca4', 0.5787805914878845),
 ('brca2', 0.5468679070472717),
 ('inherited', 0.5436486005783081),
 ('genes', 0.5065792798995972)]

#### Observaciones

Se tomaron 3 ejemplos para corroborar palabras con sus 10 más similares y en todas se logra ver la cercanía de conceptos:
- Sangre: para este caso vemos palabras como "vasos sanguíneos" y "hemoglobina", "plaquetas" y "oxígeno" que están muy relacionados con los componentes de la sangre. O por ejemplo "contraste" que refiere probablemente a las sustancias de contraste que se suministran en ciertos estudios de imágenes médicas.

- Mujeres: para el caso de mujer es donde más conceptos relacionados se ven, probablemente porque haya más enfoque en el corpus seleccionado. Vemos conceptos relacionados con el género y alteraciones que pueden afectar la enfermedad como "post y pre menospaucia", "embarazo", "lactancia" y "mama". Y, a su vez, similaritudes -sintácticas- como ser "mujer" y "hombre"

- Genes: vemos varios genes puntuales realcionados con el cancer, y por otro lado los conceptos de mutaciones o herencia, que están altamente relacionados con "mutaciones genéticas" y "herencia genética".



### 5- Analogías

Se probaron 2 ejemplos de analogías que vemos que se cumplieron:

- En el primer caso se buscó la analogía de lo que sería "cancer de mama" en la mujer, para el hombre; y se obtuvo "próstata" como resultado lo cuál es bastante acertado, en cuánto a que son órganos y localizaciones de tumores muy relacionados al género

- Para el segundo caso se buscó la anología para "vejiga", partiendo de la relación "sangre-vaso" y se obtuvo "orina" como resultado; lo cuál es correcto.

In [16]:
result = w2v_model.wv.most_similar(positive=['breast', 'men'], negative=['women'], topn=1)
print(f'La analogía para "hombre" de la relación "mujer-mama" es: {result}')

La analogía para "hombre" de la relación "mujer-mama" es: [('prostate', 0.4577924311161041)]


In [17]:
result = w2v_model.wv.most_similar(positive=['blood', 'bladder'], negative=['vessels'], topn=1)
print(f'La analogía para "vejiga" de la relación "sangre-vaso" es: {result}')

La analogía para "vejiga" de la relación "sangre-vaso" es: [('urine', 0.4657147228717804)]


### 6 - Visualizar agrupación de vectores

Se graficaron los vectores con reducción de dimensiones en 2 y 3d, y se obtuvieron los datos para la garficación con projector.tensorflow.org lo cuál mostró, incluisve con reduccipin de dimensionalidad a través de PCA o TSNE, cercanía entre los conceptos que tienen similaridad. Por ejemlo: Childohood-chlren-growth-hormones; o sunscreen-protecting-uv-sun-avoid-exposure




In [18]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [19]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [20]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [21]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)